# Anthropic Commodities Analysis Agent - Starter Implementation (built with LlamaIndex)

## Overview <br>
This system provides a comprehensive toolkit for analyzing commodities market data using the Financial Modeling Prep (FMP) API. It's designed to work within Google Colab notebooks and provides real-time commodities market analysis capabilities.

### Packages/APIs used:


1.   Anthropic API - Claude 3.5 Sonnett   
2.   LlamaIndex for orchestration
3.   Financial Modeling Prep API for commodities information
4.   News API to create up-to-date view on what's influencing commodities markets (Optional)



In [24]:
from IPython.display import Image, display
display(Image('agent-workflow-diagram.png'))

Output hidden; open in https://colab.research.google.com to view.

## 1. Install required packages and dependencies

In [1]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.3 MB/s eta 0:00:00


In [2]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

## 2. Instantiate Anthropic and Financial Modeling Prep API Keys

Step 1: Instantiate Baseline LLM to be used


*   Experiment 1: Use Claude Sonnet for tool use, code generation and parsing
*   Experiment 2: Use OpenAI o1 models to test reasoning capabilities



In [3]:
from google.colab import userdata

CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
OAI_API_KEY = userdata.get('OPENAI_API_KEY')
FMP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

In [4]:
openai_llm = OpenAI(model="gpt-4o-mini", api_key=OAI_API_KEY)

In [5]:
anthropic_llm = Anthropic(model="claude-3-5-sonnet-20240620", api_key=CLAUDE_API_KEY)

## 3. Define Classes and Tools for LLM Use

### Core Components

#### CommoditiesTracker Class
The main class that handles data fetching and analysis of commodities market data.


##### Key Methods:

###### `__init__()`
* Initializes the tracker with FMP API credentials from Colab userdata
* Sets up the base URL for API calls
* Triggers initial data fetch

###### `_fetch_data()`
* Private method that retrieves fresh commodities data from FMP API
* Converts raw JSON data into a pandas DataFrame
* Includes error handling for API requests

###### `refresh_data()`
* Public method to manually update data from the API
* Useful for getting latest market updates
* Calls `_fetch_data()` internally to refresh the DataFrame

###### `get_commodity_info(symbol: Optional[str] = None, name: Optional[str] = None)`
* Retrieves detailed information for a specific commodity
* Can search by symbol (e.g., 'GCUSD') or name (e.g., 'Gold')
* Returns comprehensive commodity data including:
  * Current price and changes
  * Daily trading ranges
  * Volume information
  * Historical averages

###### `get_top_movers(n: int = 5, by: str = 'changesPercentage')`
* Identifies top-performing commodities
* Customizable number of results (default: 5)
* Sorting metrics available:
  * Price change percentage
  * Trading volume
  * Current price
* Returns formatted list of top performers with key metrics

###### `get_market_summary()`
* Provides overall market statistics including:
  * Total number of tracked commodities
  * Count of commodities trending up/down
  * Most actively traded commodity
  * Biggest gainer of the session
  * Biggest loser of the session
* Returns structured dictionary of market metrics

###### `get_commodity_analysis(symbol: str)`
* Performs technical analysis for a specific commodity
* Technical indicators included:
  * 50-day moving average comparison
  * 200-day moving average comparison
  * Daily price range analysis
  * Yearly price range analysis
  * Volume analysis vs average
  * Price change metrics
* Returns detailed technical analysis dictionary


### 3.1 Initialize Commodities Tracker Class

In [6]:
from google.colab import userdata
import requests
import pandas as pd
from typing import Dict, List, Optional, Union
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

class CommoditiesTracker:
    def __init__(self):
        """
        Initialize the CommoditiesTracker using Google Colab userdata for API key.
        """
        self.api_key = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')
        if not self.api_key:
            raise ValueError("FMP API key not found in Colab userdata")

        self.base_url = "https://financialmodelingprep.com/api/v3"
        self.df = None
        self._fetch_data()

    def _fetch_data(self) -> None:
        """
        Fetch fresh data from the FMP API and update the internal dataframe.
        """
        url = f"{self.base_url}/quotes/commodity?apikey={self.api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            self.raw_data = response.json()
            self.df = pd.DataFrame(self.raw_data)
        except requests.exceptions.RequestException as e:
            raise Exception(f"Failed to fetch commodities data: {str(e)}")
        except Exception as e:
            raise Exception(f"Error processing commodities data: {str(e)}")

    def refresh_data(self) -> None:
        """
        Manually refresh the data from the API.
        """
        self._fetch_data()

    def get_commodity_info(self, symbol: Optional[str] = None, name: Optional[str] = None) -> Dict:
        """
        Get detailed information for a specific commodity by symbol or name.
        """
        try:
            if symbol:
                commodity = self.df[self.df['symbol'] == symbol].to_dict('records')
            elif name:
                commodity = self.df[self.df['name'].str.contains(name, case=False, na=False)].to_dict('records')
            else:
                return {"error": "Must provide either symbol or name"}

            if not commodity:
                return {"error": f"Commodity not found for symbol: {symbol} or name: {name}"}

            return commodity[0]
        except Exception as e:
            return {"error": f"Error retrieving commodity information: {str(e)}"}

    def get_top_movers(self, n: int = 5, by: str = 'changesPercentage') -> List[Dict]:
        """
        Get top n commodities by specified metric.
        """
        try:
            sorted_df = self.df.nlargest(n, by)
            return sorted_df[['symbol', 'name', by, 'price', 'change']].to_dict('records')
        except Exception as e:
            return [{"error": f"Error retrieving top movers: {str(e)}"}]

    def get_market_summary(self) -> Dict:
        """
        Get overall market summary statistics.
        """
        try:
            return {
                "total_commodities": len(self.df),
                "commodities_up": len(self.df[self.df['change'] > 0]),
                "commodities_down": len(self.df[self.df['change'] < 0]),
                "most_active": self.df.nlargest(1, 'volume')[['symbol', 'name', 'volume', 'price']].to_dict('records')[0],
                "biggest_gainer": self.df.nlargest(1, 'changesPercentage')[['symbol', 'name', 'changesPercentage', 'price']].to_dict('records')[0],
                "biggest_loser": self.df.nsmallest(1, 'changesPercentage')[['symbol', 'name', 'changesPercentage', 'price']].to_dict('records')[0]
            }
        except Exception as e:
            return {"error": f"Error retrieving market summary: {str(e)}"}

    def get_commodity_analysis(self, symbol: str) -> Dict:
        """
        Get detailed technical analysis for a specific commodity.
        """
        try:
            commodity = self.df[self.df['symbol'] == symbol].iloc[0]

            ma50_position = "above" if commodity['price'] > commodity['priceAvg50'] else "below"
            ma200_position = "above" if commodity['price'] > commodity['priceAvg200'] else "below"

            return {
                "symbol": commodity['symbol'],
                "name": commodity['name'],
                "current_price": commodity['price'],
                "technical_analysis": {
                    "ma50_analysis": f"Price is {ma50_position} 50-day MA ({commodity['priceAvg50']:.2f})",
                    "ma200_analysis": f"Price is {ma200_position} 200-day MA ({commodity['priceAvg200']:.2f})",
                    "day_range": f"${commodity['dayLow']:.2f} - ${commodity['dayHigh']:.2f}",
                    "year_range": f"${commodity['yearLow']:.2f} - ${commodity['yearHigh']:.2f}",
                    "volume_analysis": f"Current volume ({commodity['volume']}) vs Avg volume ({commodity['avgVolume']})",
                    "price_change": {
                        "value": commodity['change'],
                        "percentage": commodity['changesPercentage']
                    }
                }
            }
        except Exception as e:
            return {"error": f"Error retrieving commodity analysis: {str(e)}"}


### 3.2 Helper Functions for LLM Tool Use

- Users can create multiple helper functions for all forms of analysis - other implementations could include sentiment analysis, dataframe creation, visualizations tools, etc


In [7]:
# Function wrappers for the LLM tools
def get_commodity_info(symbol: Optional[str] = None, name: Optional[str] = None) -> Dict:
    """
    Get detailed information for a specific commodity by symbol or name.

    Args:
        symbol (str, optional): Trading symbol of the commodity (e.g., 'GCUSD' for Gold)
        name (str, optional): Name of the commodity (e.g., 'Gold')
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_commodity_info(symbol, name)
    except Exception as e:
        return {"error": f"Failed to get commodity info: {str(e)}"}

def get_top_movers(n: int = 5, metric: str = 'changesPercentage') -> List[Dict]:
    """
    Get top performing commodities by specified metric.

    Args:
        n (int): Number of commodities to return (default: 5)
        metric (str): Metric to sort by - options: 'changesPercentage', 'volume', 'price'
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_top_movers(n, metric)
    except Exception as e:
        return [{"error": f"Failed to get top movers: {str(e)}"}]

def get_market_summary() -> Dict:
    """
    Get overall commodities market summary statistics.
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_market_summary()
    except Exception as e:
        return {"error": f"Failed to get market summary: {str(e)}"}

def get_commodity_analysis(symbol: str) -> Dict:
    """
    Get detailed technical analysis for a specific commodity.

    Args:
        symbol (str): Trading symbol of the commodity (e.g., 'GCUSD' for Gold)
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_commodity_analysis(symbol)
    except Exception as e:
        return {"error": f"Failed to get commodity analysis: {str(e)}"}

## 4. Create Agentic Flow

### 4.1 Convert Functions to Tools for LLM Use

In [8]:
commodity_info_tool = FunctionTool.from_defaults(fn=get_commodity_info)
top_movers_tool = FunctionTool.from_defaults(fn=get_top_movers)
market_summary_tool = FunctionTool.from_defaults(fn=get_market_summary)
commodity_analysis_tool = FunctionTool.from_defaults(fn=get_commodity_analysis)


### 4.2 Create an Anthropic Claude Agent

In [9]:
claude_agent = FunctionCallingAgent.from_tools(
    [
        commodity_info_tool,
        top_movers_tool,
        market_summary_tool,
        commodity_analysis_tool
    ],
    llm=anthropic_llm,
    verbose=False,
    allow_parallel_tool_calls=False,
)

# Chat with an Agent

In [10]:
query= "Tell me more about Silver prices and how it's trending compared to 52-week highs/lows and the moving averages?"
response = claude_agent.chat(query)
print(str(response))

Based on the information we've gathered, here's an analysis of Silver prices and how it's trending compared to 52-week highs/lows and moving averages:

1. Current Price: Silver is currently trading at $31.485.

2. 52-Week Range:
   - 52-Week Low: $21.925
   - 52-Week High: $34.835
   - Current price is closer to the 52-week high, indicating an overall upward trend over the past year.

3. Moving Averages:
   - 50-day Moving Average: $31.7313
   - 200-day Moving Average: $28.98564
   - The current price ($31.485) is slightly below the 50-day MA but well above the 200-day MA. This suggests a short-term consolidation within a longer-term uptrend.

4. Today's Performance:
   - Silver is up 2.01% today, with a price increase of $0.621.
   - Today's range: $30.92 (low) to $31.555 (high)

5. Volume:
   - Current volume: 20,306
   - Average volume: 1,599
   - Today's volume is significantly higher than the average, indicating strong interest in Silver trading.

6. Trend Analysis:
   - Short-ter

In [11]:
query= "Which commodities have been trending favorably over the past 3-6 months?"
response = claude_agent.chat(query)
print(str(response))

Based on this information, here are the commodities that have been trending favorably over the past 3-6 months:

1. Silver (SIUSD):
   - Current price ($31.495) is well above its 200-day moving average ($28.98564)
   - It's closer to its 52-week high ($34.835) than its low ($21.925)
   - The 50-day moving average ($31.7313) is above the 200-day moving average, indicating a positive trend

2. Crude Oil (CLUSD):
   - While it's showing strong recent gains (up 2.85% today), its longer-term trend is less clear
   - Current price ($70.04) is below both its 50-day ($70.381) and 200-day ($76.52755) moving averages
   - However, it's closer to its 52-week high ($87.67) than its low ($65.27), suggesting potential for recovery

3. Copper (HGUSD):
   - Current price ($4.2075) is slightly below both its 50-day ($4.29916) and 200-day ($4.30147) moving averages
   - It's closer to its 52-week high ($5.1985) than its low ($3.686)
   - The moving averages are very close, suggesting a possible consolid

In [12]:
query= "Which metal commodities have been trending favorably? Present your findings in a table format"
response = claude_agent.chat(query)
print(str(response))

Now that we have the information for these metal commodities, let's present the findings in a table format:

| Metal     | Symbol | Current Price | % Change | 52-Week Low | 52-Week High | 50-Day MA | 200-Day MA | Trend Analysis |
|-----------|--------|---------------|----------|-------------|--------------|-----------|------------|-----------------|
| Gold      | GCUSD  | $2,671.20     | +0.48%   | N/A         | N/A          | N/A       | N/A        | Insufficient data for long-term trend analysis |
| Silver    | SIUSD  | $31.515       | +2.11%   | $21.925     | $34.835      | $31.7313  | $28.98564  | Bullish: Above both 50-day and 200-day MA |
| Copper    | HGUSD  | $4.2085       | +1.89%   | $3.686      | $5.1985      | $4.29916  | $4.30147   | Neutral: Slightly below both MAs, but close |
| Platinum  | PLUSD  | $960.00       | +1.01%   | $871.4      | $1,084.6     | $982.292  | $964.836   | Neutral: Below 50-day MA, but above 200-day MA |
| Palladium | PAUSD  | $987.00       | -0.82

In [ ]:
query= "Compare gold and silver commodities - do fundamental analysis?"
response = claude_agent.chat(query)
print(str(response))

Now, let's compare gold and silver commodities based on this information:

1. Price and Performance:
   - Gold: $2,756.80 per ounce, up 0.27% today
   - Silver: $32.86 per ounce, up 0.20% today

   Both metals are showing positive performance today, with gold slightly outperforming silver.

2. Year-to-Date Performance:
   - Gold: Currently near its yearly high of $2,789.00 (52-week range: $1,932.60 - $2,789.00)
   - Silver: Also near its yearly high of $34.84 (52-week range: $21.93 - $34.84)

   Both metals have shown strong performance over the past year, with current prices near their yearly highs.

3. Moving Averages:
   - Gold: Price is above both 50-day MA ($2,616.87) and 200-day MA ($2,357.10)
   - Silver: Price is above both 50-day MA ($31.07) and 200-day MA ($28.07)

   Both metals are trading above their key moving averages, indicating bullish trends.

4. Volume:
   - Gold: Current volume (50,906) is significantly higher than average volume (614)
   - Silver: Current volume (1

In [ ]:
query= "Create a fundamental analysis on Platinum"
response = claude_agent.chat(query)
print(str(response))

Based on this information, let's create a fundamental analysis for Platinum:

Fundamental Analysis of Platinum (PLUSD)

1. Price and Performance:
   - Current Price: $1,010.40 per ounce
   - Daily Change: Up 1.08% ($10.80)
   - Year Range: $838.60 - $1,084.60
   - The current price is closer to the yearly high, indicating a generally positive trend over the past year.

2. Technical Indicators:
   - Moving Averages: The price is above both the 50-day ($979.43) and 200-day ($958.30) moving averages, suggesting a bullish trend in both short and long-term timeframes.
   - Volume: Current volume (6,172) is significantly higher than the average volume (215), indicating increased interest and trading activity.

3. Supply and Demand Factors:
   a. Industrial Demand:
      - Platinum has significant industrial applications, particularly in the automotive industry for catalytic converters.
      - It's also used in jewelry, electronics, and chemical processing.
      - The price increase might s